In [1]:
import pandas as pd
import numpy as np
import time
import os
import urllib.request
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import pickle
import re

from selenium.common.exceptions import StaleElementReferenceException

chrome_options = Options()
chrome_options.add_argument('--headless')  # 웹 브라우저를 표시하지 않고 백그라운드에서 실행
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

# 크롤링 전, solved.ac 문제 크롤링 파일 정제.

In [2]:
df = pd.read_csv('../DataSet/solved.ac_문제_크롤링.csv')
df

,problem_number,problem_name,problem_algorithm,problem_solve_people_number,problem_avg_try,problem_tier
0,1010,다리 놓기,"['조합론', '다이나믹 프로그래밍', '수학']",32526,2.07,6
1,1094,막대기,"['비트마스킹', '수학']",17655,1.33,6
2,1181,단어 정렬,"['정렬', '문자열']",49263,2.48,6
3,1193,분수찾기,"['구현', '수학']",47756,1.94,6
4,1251,단어 나누기,"['브루트포스 알고리즘', '구현', '정렬', '문자열']",3752,2.16,6
...,...,...,...,...,...,...
16307,29205,마라톤,"['벨만–포드', '볼록 껍질을 이용한 최적화', '데이크스트라', '그래프 이론']",12,6.75,22
16308,29789,The Ties That Guide Us,"['센트로이드', '해 구성하기', '깊이 우선 탐색', '그래프 이론', '그래프...",1,15.00,22
16309,29949,Tree Isomorphism,"['트리', '트리 동형 사상']",4,4.25,22
16310,30029,도미노 수열,"['자료 구조', '느리게 갱신되는 세그먼트 트리', '세그먼트 트리', '스플레이...",7,1.57,22


In [3]:
import re

#푼 사람이 적은 문제는 제외.

standard_problem_count = 0

df = df[df['problem_solve_people_number'] >= standard_problem_count]

#한국어, 영어 외의 다른 나라 언어 제외
def is_korean_or_english(input_string):
    # 정규 표현식을 사용하여 문자열이 한글 또는 영어로만 이루어져 있는지 확인
    pattern = re.compile("^[가-힣a-zA-Z0-9\\-,.?!';()’,;/×+~&^\\s:→]*[가-힣a-zA-Z0-9\\-,.?!';()’,;/×+~&^:→]+[가-힣a-zA-Z0-9\\-,.?!';()’,;/×+~&^\\s:→]*$")
    return bool(pattern.match(input_string))

df_c = df[~df['problem_name'].apply(is_korean_or_english)]
df = df[df['problem_name'].apply(is_korean_or_english)]

#정규표현식으로 못 거르는 좋은 문제들 수동으로 추가.
number_list = [
    4378, 20546, 29727, 18869, 20500, 23562, 25312, 23370, 10529, 10772, 25585, 13172, 14497,
    16696, 17281, 23031, 29737, 2733, 14786, 15487, 19535, 20440, 27314, 4893, 13031, 20442,
    1016, 3954, 11689, 13173, 14679, 15807, 16163, 27320, 1502, 2978, 28704, 11475, 20307,
    24271, 26038, 27169, 18765, 20441, 20498, 21863, 22901, 28398, 1557, 13705, 13926, 19540, 26036,
    3685, 26356, 3071, 26257, 17290, 20171, 17203
]

df_c = df_c[df_c['problem_number'].isin(number_list)]

df = pd.concat([df, df_c], axis=0)
df = df.reset_index(drop=True)
df

,problem_number,problem_name,problem_algorithm,problem_solve_people_number,problem_avg_try,problem_tier
0,1010,다리 놓기,"['조합론', '다이나믹 프로그래밍', '수학']",32526,2.07,6
1,1094,막대기,"['비트마스킹', '수학']",17655,1.33,6
2,1181,단어 정렬,"['정렬', '문자열']",49263,2.48,6
3,1193,분수찾기,"['구현', '수학']",47756,1.94,6
4,1251,단어 나누기,"['브루트포스 알고리즘', '구현', '정렬', '문자열']",3752,2.16,6
...,...,...,...,...,...,...
15596,13705,Ax+Bsin(x)=C,"['임의 정밀도 / 큰 수 연산', '이분 탐색', '수학', '수치해석']",489,25.01,21
15597,13926,"gcd(n, k) = 1","['오일러 피 함수', '수학', '밀러–라빈 소수 판별법', '정수론', '폴라드...",505,2.94,21
15598,19540,인버스 ㄷㄷㄷㅈ,"['해 구성하기', '트리']",67,2.09,21
15599,26036,제곱 ㄱㄱ,"['임의 정밀도 / 큰 수 연산', '해 구성하기', '수학', '정수론']",21,1.52,21


In [4]:
total_problem_number_list = list(df['problem_number'])
total_problem_name_list = list(df['problem_name'])
total_problem_tier_list = list(df['problem_tier'])
total_problem_algorithm_list = list(df['problem_algorithm'])
total_problem_avg_try_list = list(df['problem_avg_try'])

In [2]:
#처음 크롤링할 때 리스트 초기화 코드.
#중간에 이거 실행하고 다시 크롤링 코드 실행하거나 껐다가 크롤링코드 실행하면 pickle파일 다 날라간다!  주의!!! 
#봉인해두고 실행해야할때만 열어서 실행 

total_crawling_problem_number_list = []
total_problem_time_condition_list = []
total_problem_memory_condition_list = []
total_problem_submission_count_list = []
total_problem_answer_submission_count_list = []
total_problem_answered_people_count_list = []
total_problem_correct_rate_list = []
remove_problem_list = []
total_english_problem_list = []


In [6]:
#에러 생겨서 다시할 때는 pickle에 저장된 것 부터 시작. 

file_name = "DB_problem_table.pkl"

with open(file_name, 'rb') as file:
    loaded_dicts = pickle.load(file)

total_crawling_problem_number_list = loaded_dicts['total_crawling_problem_number_list']
total_problem_time_condition_list = loaded_dicts['total_problem_time_condition_list']
total_problem_memory_condition_list = loaded_dicts['total_problem_memory_condition_list']
total_problem_submission_count_list = loaded_dicts['total_problem_submission_count_list']
total_problem_answer_submission_count_list = loaded_dicts['total_problem_answer_submission_count_list']
total_problem_answered_people_count_list = loaded_dicts['total_problem_answered_people_count_list']
total_problem_correct_rate_list = loaded_dicts['total_problem_correct_rate_list']
remove_problem_list = loaded_dicts['remove_problem_list']
total_english_problem_list = loaded_dicts['total_english_problem_list']

In [13]:
#껐다가 크롤링 코드 실행하면 pickle파일 다 날라간다!  주의!!! 
#봉인해두고 실행해야할 때만 열어서 실행 

for n in total_problem_number_list:

    if((n in total_crawling_problem_number_list) or (n in total_english_problem_list)):
        continue

    problem_url = 'https://www.acmicpc.net/problem/' + str(n)
    print(problem_url)
   
    driver = webdriver.Chrome('chromedriver.exe', options=chrome_options)
    driver.get(problem_url)

    problem_description = driver.find_element_by_css_selector('#problem_description')
    
    if(not any(char >= '가' and char <= '힣' for char in problem_description.text)):
        total_english_problem_list.append(n)
        print('영어 문제')
        continue


    problem_info_object = driver.find_element_by_css_selector('#problem-info')
    problem_info_list = [float(pio) if '.' in pio else int(pio) for pio in re.findall(r'\d+\.\d+|\d+', problem_info_object.text.split('\n')[1])]
    
    if(len(problem_info_list)!=6):
        
        remove_problem_list.append(n)
        r_index = total_problem_number_list.index(n)
        
        total_problem_number_list.pop(r_index)
        total_problem_name_list.pop(r_index)
        total_problem_tier_list.pop(r_index)
        total_problem_algorithm_list.pop(r_index)
        total_problem_avg_try_list.pop(r_index)
        driver.quit()

        continue

    total_problem_time_condition_list.append(problem_info_list[0])
    total_problem_memory_condition_list.append(problem_info_list[1])
    total_problem_submission_count_list.append(problem_info_list[2])
    total_problem_answer_submission_count_list.append(problem_info_list[3])
    total_problem_answered_people_count_list.append(problem_info_list[4])
    total_problem_correct_rate_list.append(problem_info_list[5])
    total_crawling_problem_number_list.append(n)

    
    lists_to_save = {
        'total_crawling_problem_number_list': total_crawling_problem_number_list,
        'total_problem_time_condition_list': total_problem_time_condition_list,
        'total_problem_memory_condition_list': total_problem_memory_condition_list,
        'total_problem_submission_count_list': total_problem_submission_count_list,
        'total_problem_answer_submission_count_list': total_problem_answer_submission_count_list,
        'total_problem_answered_people_count_list': total_problem_answered_people_count_list,
        'total_problem_correct_rate_list': total_problem_correct_rate_list,
        'remove_problem_list': remove_problem_list,
        'total_english_problem_list': total_english_problem_list
    }

    file_name = "DB_problem_table.pkl"
    with open(file_name, 'wb') as file:
        pickle.dump(lists_to_save, file) 
    
    driver.quit()


https://www.acmicpc.net/problem/24921
영어 문제
https://www.acmicpc.net/problem/24940
https://www.acmicpc.net/problem/24980
영어 문제
https://www.acmicpc.net/problem/25047
https://www.acmicpc.net/problem/25049
https://www.acmicpc.net/problem/25056
영어 문제
https://www.acmicpc.net/problem/25077
영어 문제
https://www.acmicpc.net/problem/25115
영어 문제
https://www.acmicpc.net/problem/25174
https://www.acmicpc.net/problem/25188
https://www.acmicpc.net/problem/25197
https://www.acmicpc.net/problem/25208
https://www.acmicpc.net/problem/25237
영어 문제
https://www.acmicpc.net/problem/25240
https://www.acmicpc.net/problem/25255
영어 문제
https://www.acmicpc.net/problem/25279
영어 문제
https://www.acmicpc.net/problem/25307
https://www.acmicpc.net/problem/25309
https://www.acmicpc.net/problem/25332
https://www.acmicpc.net/problem/25339
https://www.acmicpc.net/problem/25341
https://www.acmicpc.net/problem/25368
https://www.acmicpc.net/problem/25420
https://www.acmicpc.net/problem/25435
영어 문제
https://www.acmicpc.net/problem/25

In [14]:
print(len(total_crawling_problem_number_list))
print(len(total_problem_time_condition_list))
print(len(total_problem_memory_condition_list))
print(len(total_problem_submission_count_list))
print(len(total_problem_answer_submission_count_list))
print(len(total_problem_answered_people_count_list))
print(len(total_problem_correct_rate_list))
print(len(total_problem_number_list))


print(total_crawling_problem_number_list)
print(total_problem_time_condition_list[-1])
print(total_problem_memory_condition_list[-1])
print(total_problem_submission_count_list[-1])
print(total_problem_answer_submission_count_list[-1])
print(total_problem_answered_people_count_list[-1])
print(total_problem_correct_rate_list[-1])
print(remove_problem_list)
#print(total_problem_number_list)


6941
6941
6941
6941
6941
6941
6941
15601
[1010, 1094, 1181, 1193, 1251, 1308, 1312, 1316, 1340, 1343, 1380, 1384, 1402, 1417, 1418, 1427, 1436, 1439, 1475, 1476, 1531, 1543, 1652, 1676, 1730, 1769, 1789, 1817, 1996, 2018, 2057, 2097, 2161, 2167, 2238, 2303, 2508, 2535, 2545, 2563, 2622, 2628, 2635, 2669, 2714, 2740, 2751, 2822, 2828, 2890, 2891, 2941, 2993, 3049, 3054, 3060, 3135, 3230, 3699, 3711, 3724, 3863, 4108, 4158, 4335, 4383, 4446, 4659, 4673, 4821, 4884, 5046, 5347, 5426, 5462, 5555, 5635, 5648, 5671, 5800, 6417, 6463, 6504, 6550, 7568, 7656, 7785, 8611, 8892, 8979, 9037, 9196, 9237, 9324, 9358, 9625, 9626, 9655, 9733, 10384, 10431, 10469, 10709, 10814, 10815, 10826, 10867, 10914, 11004, 11008, 11059, 11068, 11116, 11256, 11292, 11504, 11536, 11576, 11637, 11650, 11651, 11723, 11728, 11866, 11931, 11971, 12033, 12034, 12437, 12760, 12787, 12871, 13119, 13133, 13241, 13301, 13333, 13717, 13909, 14490, 14492, 14493, 14582, 14606, 14608, 14646, 14654, 14655, 14709, 14717, 14721, 

In [15]:
#데이터 순서 맞춰서 정렬후 데이터프레임으로 변환 후 저장.

total_problem_number_list = total_crawling_problem_number_list

def get_problem_values_in_order(df, total_problem_number_list, vaules):

    result_list = []

    for problem_number in total_problem_number_list:
        problem_vaules = df.loc[df['problem_number'] == problem_number, vaules].values
        if len(problem_vaules) > 0:
            result_list.append(problem_vaules[0])

    return result_list


total_problem_name_list = get_problem_values_in_order(df, total_problem_number_list, 'problem_name')
total_problem_tier_list = get_problem_values_in_order(df, total_problem_number_list, 'problem_tier')
total_problem_algorithm_list = get_problem_values_in_order(df, total_problem_number_list, 'problem_algorithm')
total_problem_avg_try_list = get_problem_values_in_order(df, total_problem_number_list, 'problem_avg_try')

#print(total_problem_name_list)
print(len(total_problem_name_list))
#print(total_problem_tier_list)
print(len(total_problem_tier_list))
#print(total_problem_algorithm_list)
print(len(total_problem_algorithm_list))
#print(total_problem_avg_try_list)
print(len(total_problem_avg_try_list))




6941
6941
6941
6941


In [16]:

data = {'problem_number': total_problem_number_list, 
        'problem_name': total_problem_name_list,
        'problem_tier': total_problem_tier_list, 
        'problem_algorithm': total_problem_algorithm_list, 
        'problem_time_condition': total_problem_time_condition_list, 
        'problem_memory_condition': total_problem_memory_condition_list, 
        'problem_submission_count': total_problem_submission_count_list, 
        'problem_answer_submission_count': total_problem_answer_submission_count_list,
        'problem_answered_people_count': total_problem_answered_people_count_list,
        'problem_correct_rate': total_problem_correct_rate_list,
        'problem_avg_try' : total_problem_avg_try_list
    }

result_df = pd.DataFrame(data)
result_df

,problem_number,problem_name,problem_tier,problem_algorithm,problem_time_condition,problem_memory_condition,problem_submission_count,problem_answer_submission_count,problem_answered_people_count,problem_correct_rate,problem_avg_try
0,1010,다리 놓기,6,"['조합론', '다이나믹 프로그래밍', '수학']",0.5,128,88923,41540,33782,48.084,2.07
1,1094,막대기,6,"['비트마스킹', '수학']",2.0,128,29308,21683,18316,75.244,1.33
2,1181,단어 정렬,6,"['정렬', '문자열']",2.0,256,163610,68439,51305,40.323,2.48
3,1193,분수찾기,6,"['구현', '수학']",0.5,256,113582,57335,49269,51.490,1.94
4,1251,단어 나누기,6,"['브루트포스 알고리즘', '구현', '정렬', '문자열']",2.0,128,11073,4633,3992,46.007,2.16
...,...,...,...,...,...,...,...,...,...,...,...
6936,13705,Ax+Bsin(x)=C,21,"['임의 정밀도 / 큰 수 연산', '이분 탐색', '수학', '수치해석']",2.0,512,17330,1173,508,4.026,25.01
6937,13926,"gcd(n, k) = 1",21,"['오일러 피 함수', '수학', '밀러–라빈 소수 판별법', '정수론', '폴라드...",2.0,512,2257,668,536,33.333,2.94
6938,19540,인버스 ㄷㄷㄷㅈ,21,"['해 구성하기', '트리']",2.0,1024,211,101,67,47.857,2.09
6939,26036,제곱 ㄱㄱ,21,"['임의 정밀도 / 큰 수 연산', '해 구성하기', '수학', '정수론']",1.0,512,71,33,22,64.706,1.52


In [ ]:
result_df.to_csv('Baekjoon_문제_크롤링.csv',index=False)